In [40]:
def count_apperance(result_list,data):
    c = []
    for i in result_list:
        c.append((i,0))

    for i in range(len(c)):
        for j in range(len(data)):
            if c[i][0].issubset(data[j][1]):
                c[i] = (c[i][0],c[i][1]+1)
    return c

In [41]:
from itertools import combinations

def prune(k, result_list, new):
    for i in result_list:
        subsets = set(combinations(i,k))
        for j in subsets:
            j = set(j)
            if j not in new:
                result_list.remove(i)
                break
    return result_list

def self_join(k, c, support):
    new = [i[0] for i in c]
    result_list = []
    for i in range(len(new)):
        for j in range(i+1, len(new)):
                    temp1 = list(new[i])
                    temp2 = list(new[j])
                    if(temp1[0:k] == temp2[0:k]):
                        joined = set(temp1 + temp2)
                        result_list.append(joined)
                        
                        #second step of pruning is still missing see slide 39
    output = prune(k+1, result_list, new)
    return output




c = [({1,2,3},2),({1,2,4},2),({1,3,4},2),({1,3,5},2),({2,3,4},2)]

result_list = self_join(2, c, 2)
print(result_list)

[{1, 2, 3, 4}]


In [42]:
from itertools import combinations, product

import csv

def important_print(more, c):
    for i in c:
        if i[1] == more:
            print(i)

def numberToName(c, names):
    output = []
    for i in range(len(c)):          
        new = set()
        for number in c[i][0]:
            new.add(names[number])
        output.append((new,c[i][1]))
    return output

def getConfidence(old, new):
    output=[]
    for i in old:
        for j in new:
            if i[0].issubset(j[0]):
                oldsup = i[1]
                newsup = j[1]
                conf = newsup/oldsup
                output.append((i[0],j[0]-i[0],conf, newsup))
    return output

In [43]:


file_path = 'aap.txt'  # Replace 'your_file.txt' with the path to your text file

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read all lines from the file
    lines = file.readlines()

# Extract names from the lines, skipping the index at the beginning
products = [line.split()[1].strip() for line in lines]


database = []
with open('myDataFile.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        database.append(row)

support = 49

data = []
for i in range(len(database)):
    data.append((i,set()))

# makes the database to workable format
for i in range(len(database)):
    for j in range(len(database[i])):
        if database[i][j] == 't':
           data[i][1].add(j)   

# first round to initialize c
k = 1
c = []
for i in range(len(database[0])):
    c.append(({i},0))

for i in range(len(c)):
    for j in range(len(data)):
        if c[i][0].issubset(data[j][1]):
            c[i] = (c[i][0],c[i][1]+1)


k = 0
# c is previous list with k = x counted is new list with k = x+1. before end c becomes only good relations
while c:
    result_list = self_join(k,c, support)
    counted = count_apperance(result_list, data)
    counted = [item for item in counted if item[1] >= support]

    
    confidence = getConfidence(c, counted)
    
    print("confidence")
    print(confidence)
    c = counted
    if c:
        names = numberToName(c, products)
        
    k+=1

print(names)


confidence
[({1}, {13}, 0.2188449848024316, 72), ({1}, {103}, 0.24316109422492402, 80), ({1}, {123}, 0.19148936170212766, 63), ({1}, {139}, 0.22796352583586627, 75), ({1}, {158}, 0.14893617021276595, 49), ({1}, {167}, 0.22188449848024316, 73), ({7}, {103}, 0.41379310344827586, 72), ({7}, {166}, 0.5229885057471264, 91), ({9}, {13}, 0.1182170542635659, 61), ({9}, {16}, 0.11046511627906977, 57), ({9}, {31}, 0.16085271317829458, 83), ({9}, {50}, 0.11434108527131782, 59), ({9}, {65}, 0.09689922480620156, 50), ({9}, {89}, 0.1182170542635659, 61), ({9}, {96}, 0.12209302325581395, 63), ({9}, {103}, 0.375968992248062, 194), ({9}, {106}, 0.12015503875968993, 62), ({9}, {112}, 0.14534883720930233, 75), ({9}, {123}, 0.2596899224806202, 134), ({9}, {124}, 0.3313953488372093, 171), ({9}, {131}, 0.1065891472868217, 55), ({9}, {139}, 0.15503875968992248, 80), ({9}, {158}, 0.14534883720930233, 75), ({9}, {162}, 0.12790697674418605, 66), ({9}, {166}, 0.4050387596899225, 209), ({9}, {167}, 0.222868217054